<a href="https://colab.research.google.com/github/PradipNichite/Youtube-Tutorials/blob/main/Youtube_Text_Classification_using_sentence_embedding_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Resources:

*   https://www.sbert.net/index.html
*   https://www.sbert.net/docs/pretrained_models.html




In [1]:
!pip install -U sentence-transformers -q

In [2]:
import pandas as pd
import numpy as np
np.random.seed(2022)

In [3]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2')

In [4]:
data = pd.read_csv("/content/drive/MyDrive/Content Creation/Youtube Tutorials/datasets/toxic_commnets_500.csv",error_bad_lines=False, engine="python")

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


In [5]:
data.head()

,comment_text,toxic
0,Your knowledge of Hebrew? Thank god that Jossi...,0
1,==Tablighi Jamaat and allegations of terrorism...,0
2,"""\nWell it's understandable not wanting the in...",0
3,"""\n\n Please stop adding nonsense to Wikipedia...",0
4,"Saint Petersburg\nPlease, leave the sister cit...",0


In [6]:
import spacy
import string
nlp = spacy.load("en_core_web_sm")
stop_words = nlp.Defaults.stop_words
print(stop_words)

{'thru', 'almost', 'sometimes', 'hereby', 'upon', 'thus', 'own', 'nevertheless', 'those', 'is', 'sometime', 'our', 'since', 'she', 'afterwards', 'much', 'its', 'meanwhile', 'ever', 'here', 'only', 'either', 'first', 'n’t', 'five', 'their', 'towards', 'twelve', 'any', 'your', 'empty', 'bottom', 'hers', '‘re', 'seems', 'although', 'such', 'thereafter', 'out', 'enough', 'by', 'again', 'became', 'thence', 'make', 'throughout', 'you', 'around', 'nobody', 'over', 'each', 'through', "'d", 'us', 'when', 're', 'being', 'eight', 'too', 'please', 'were', 'others', 'he', 'why', 'an', 'everyone', 'say', 'yourself', 'which', 'from', 'but', 'anyone', 'four', 'none', 'give', 'herein', 'whereupon', 'whoever', 'regarding', 'has', 'name', 'how', 'been', 'less', 'using', 'behind', 'ca', 'for', 'amongst', 'call', 'have', 'indeed', 'was', 'perhaps', 'yet', 'into', 'until', 'whence', 'would', 'thereupon', 'will', 'everything', 'somehow', 'we', 'whole', 'becoming', 'below', 'nothing', 'might', 'who', 'during'

In [7]:
punctuations = string.punctuation
print(punctuations)

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


In [8]:
# Creating our tokenizer function
def spacy_tokenizer(sentence):
    # Creating our token object, which is used to create documents with linguistic annotations.
    doc = nlp(sentence)



    # print(doc)
    # print(type(doc))

    # Lemmatizing each token and converting each token into lowercase
    mytokens = [ word.lemma_.lower().strip() for word in doc ]

    # print(mytokens)

    # Removing stop words
    mytokens = [ word for word in mytokens if word not in stop_words and word not in punctuations ]
    
    sentence = " ".join(mytokens)
    # return preprocessed list of tokens
    return sentence

In [16]:
list_ = ["happy" , "apple"]
"-".join(list_)

'happy-apple'

In [14]:
spacy_tokenizer("I am Eating")

'eat'

In [12]:
data['tokenize'] = data['comment_text'].apply(spacy_tokenizer)

In [17]:
data.head()

,comment_text,toxic,tokenize
0,Your knowledge of Hebrew? Thank god that Jossi...,0,knowledge hebrew thank god jossi speak hebrew ...
1,==Tablighi Jamaat and allegations of terrorism...,0,tablighi jamaat allegation terrorism== add art...
2,"""\nWell it's understandable not wanting the in...",0,understandable want infobox overfill info fami...
3,"""\n\n Please stop adding nonsense to Wikipedia...",0,stop add nonsense wikipedia consider vandalism...
4,"Saint Petersburg\nPlease, leave the sister cit...",0,saint petersburg leave sister city ill try fin...


In [18]:
data['embeddings'] = data['tokenize'].apply(model.encode)

In [19]:
data.head()

,comment_text,toxic,tokenize,embeddings
0,Your knowledge of Hebrew? Thank god that Jossi...,0,knowledge hebrew thank god jossi speak hebrew ...,"[0.040585447, 0.10845654, 0.016405603, -0.0097..."
1,==Tablighi Jamaat and allegations of terrorism...,0,tablighi jamaat allegation terrorism== add art...,"[0.013790878, 0.06591222, -0.07681706, -0.0021..."
2,"""\nWell it's understandable not wanting the in...",0,understandable want infobox overfill info fami...,"[0.012403863, 0.04393697, 0.052039493, 0.02465..."
3,"""\n\n Please stop adding nonsense to Wikipedia...",0,stop add nonsense wikipedia consider vandalism...,"[0.020707756, 0.03334088, 0.0177352, -0.036466..."
4,"Saint Petersburg\nPlease, leave the sister cit...",0,saint petersburg leave sister city ill try fin...,"[0.04717972, 0.029359365, -0.11119904, 0.07145..."


In [20]:
X = data['embeddings'].to_list()
y = data['toxic'].to_list()

In [22]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,stratify=y)

In [23]:
from sklearn.linear_model import LogisticRegression
LR = LogisticRegression()
LR.fit(X_train,y_train)

LogisticRegression()

In [24]:
from sklearn import metrics
predicted = LR.predict(X_test)
print("Logistic Regression Accuracy:",metrics.accuracy_score(y_test, predicted))
print("Logistic Regression Precision:",metrics.precision_score(y_test, predicted))
print("Logistic Regression Recall:",metrics.recall_score(y_test, predicted))

Logistic Regression Accuracy: 0.88
Logistic Regression Precision: 0.8725490196078431
Logistic Regression Recall: 0.89
